---
# **Title**:TRABAJO AP. DDPG TD(3) (Gradiente de política determinista profunda (TD3) de doble retardo. Entornos de estados continuos: Humanoide). + Conexión Chatgpt (o similar)  + modelo dalle (similr)

Versión 3: Modificamos la estrategia de Exploracion Vs Explotción. Haremos un entrenamiento en dos partes.

1 ) Recorremos el entorno en busca de observaciones y acciones asociadas subóptmas. Rellenamos la memoria de repetición con estas observciones.

2) Entrenamos de forma normal tal que cuando vayamos a seleccionar la acción tenemos varias fases.:

  a ) primera fase donde usamos las acciones almacenadas en el replaybuffer.

  b ) segunda fase de esplotación de las acciones de la política pero con una probabilidad de realizar mñas explotaciones, tal que eta probabilidad va en decaimiento a medida que hacemos más pasos.

Nota:
  - Para seleccionar la acción dentro de la memoria de repetición en la primera fase, lo que hacemos es buscar aquella transición que maas se aprezca a la observación (estado) del momento actual.

  - Cuando añadimos una nueva transición a la memoria de repetición si el buffer esta lleno, lo que hacemos es buscar la transición con menor reward y la intercambiamos con la nueva transición.

# ESTA VERSIÓN ES LA PEOR DE TODAS. SÓLO CONSIGUE SOBREAJUSTE TANTO DEL CRíTICO COMO DEL ACTOR AL CERRARSE EXCLUSIVAMENTE A UNAS POCAS ACCIONES (con reward >0)

# **Author**: José Javier Gutiérrez Gil
# **Date**: 2024-02-18
# ***Univeridad de Valencia. Grado de Ciencia de Datos***
---

In [2]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/td3')

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/td3')

In [5]:
!pip install pybullet
!pip install gym==0.22.0 # Versión más actual que contiene la calse Monitor y así poder crear los videos del entrenamiento. Al monos en la 0.23 me da error

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.22.0-py3-none-any.whl size=708364 sha256=df87f9b58b6eafcd6610a1f35f4d0b04074b0cc247bfd90bb7d1973e03082c01
  Stored in directory: /root/.cache/pip/wheels/42/e8/e8/6dfbc92a1dcd76c1a5e2bb982750fd6b7e792239f46039e6b1
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


# **Step 0:** Cargamos las funciones necesarias de nuestra libreria y de python

In [6]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#####
import gym
import pybullet_envs
from gym import wrappers
##
import torch
import torch.nn as nn
#####
from TD3 import TD3, ReplayBufferPos
from TD3 import created_models_directory, mkdir
from TD3 import evaluate_train_policy, evaluate_policy
from TD3 import noisy_action_wrapper, save_env, load_env, create__metrics_imagen
from TD3 import serialize_object, lists_to_serializable_object, serialize_training


# **Step 1:** Inicializamos los hiperparámetros del modelo e implementación

In [7]:

env_name            =  "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed                = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
initial_memory_prob = 0.1      # Define la probabilidad inicial de tomar una acción de la memoria de repetición
max_start_steps     = 1e4      # Define el número máximo de pasos iniciales
#start_steps         = 1e4#1e4  --123-- lo modificamos por un decaimiento según los pasos de entrenamiento # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency      = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps       = 1e6      # Número máximo de iteraciones/timesteps permitidos
save_models         = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
max_explore_noise   = 0.1      # Desviación estándar del ruido gaussiano utilizado para la exploración...--123-- CaMBIO  0.01 por 0.2 y creo un current_noise_explore dependiendo del steo en el que estamos
batch_size          = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma               = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq  = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise        = 0.2     # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip          = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq         = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

# **Step 2:** Creamos los directorios y cargamos el entorno de trabajo

In [8]:
#Creamos los directorios de almacenamento de resultados y métricas
file_model_name = created_models_directory (env_name, seed, save_models, "v3_1")

# Cargamos el entorno sobre el cual ejecutaremso el modelo DDPG TD3
###############################################################################
##  CARGAMOS EL ENTORNO HUMANOIDE V0 Y LE INTRODUCIMOS EL WRAPPER PARA AÑADIR RUIDO A
##  LAS ACCIONES OBTENIDAS POR EXPLORACIÓN.
################################################################################
env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
env = noisy_action_wrapper (env, noise_level = policy_noise) #le agregamos ruido a la accion obtenida del entorno para darle ms estabilidad al entrenamiento
# Fijamos la semilla y obtenemos información del entorno (Estados, acciones)
env.seed (seed)
torch.manual_seed (seed)
np.random.seed (seed)

state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float(env.action_space.high [0])

#Creamos los directorios de trabajo donde guardará los videos del entrenamiento
work_dir          = mkdir ('exp', 'brs')
monitor_dir       = mkdir (work_dir, 'monitor')

save_env_vid      = False
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  env.reset()


---------------------------------------
Fichero de los modelos entrenados: TD3_HumanoidBulletEnv-v0_0_v3_1
---------------------------------------


# **Step 3:** Proceso de entrenamiento : por semiimitación

En lugar de seguir la tradicional división entre las fases de exploración y explotación durante el entrenamiento, optamos por dos etapas diferenciadas. En la primera etapa, nos enfocaremos exclusivamente en buscar transiciones en el entorno que resulten en recompensas positivas. Esta fase se llevará a cabo durante un número fijo de pasos hasta que hayamos recopilado un replay buffer completo con transiciones positivas. Una vez completada esta fase y con la memoria de repetición llena de transiciones subóptimas, procederemos con el entrenamiento normal. Aunque nos centramos en la explotación, el proceso de añadir ruido a las acciones durante esta fase puede considerarse una forma de semi-exploración. Hemos observado que esta estrategia nos ha brindado mayores beneficios en comparación con la exploración aleatoria en un entorno con un espacio de acciones infinitas, ya que nos ayuda a evitar mínimos locales y a mejorar la calidad del movimiento del agente.

# Step 3.1: Etapa de explotación y busqueda en el entorno Humanoide

Creamos la memoria de repetición y almacenamos directamente tantas transiciones como encuentre con una recompensa positiva. Hasta el máximo de la capacidad de la memoria de repetición (ReplayBufferPos)

In [9]:
import gym
import numpy as np

max_steps = max_timesteps  # Número máximo de pasos
positive_rewards = []

total_steps       = 0
episodeo          = 0
# Nos creamos la memoria de repetición de experiencias
max_cap = int(2e4)
replay_buff_pos =  ReplayBufferPos(max_capacity = max_cap)# Notar que la máxima capacidad de la memoria por defecto es: max_capacity = 1e4

max_iter = 3 * max_steps
while  total_steps < max_iter:
    obs = env.reset()  # Reiniciar el entorno y obtener el estado inicial
    done = False
    while not done:
        action = env.action_space.sample()  # Tomar una acción aleatoria
        next_obs, reward, done, _ = env.step(action)  # Realizar la acción y observar el resultado

        # Almacenar la transición (estado inicial, estado final, acción, recompensa)
        transition = (obs, next_obs, action, reward, done)

        print(f'Total de pasos: {total_steps}/{max_iter} - Recompensa: {reward}')

        # Verificar si la recompensa es positiva y almacenarla en la lista
        if reward > 0:
            print (reward)

            positive_rewards.append (reward)
            replay_buff_pos.add (transition)

        obs = next_obs
        total_steps += 1
        if total_steps >= max_steps:
            break

    # Verificar si todas las transiciones tienen recompensas positivas
    if len (positive_rewards) == max_start_steps:
        print("Todas las transiciones tienen recompensas positivas. Deteniendo el entrenamiento.")
        break

Se han truncado las últimas 5000 líneas del flujo de salida.
Total de pasos: 2995840/3000000.0 - Recompensa: -1.4337121669480068
Total de pasos: 2995841/3000000.0 - Recompensa: -3.1581809364769287
Total de pasos: 2995842/3000000.0 - Recompensa: -0.8638081098753155
Total de pasos: 2995843/3000000.0 - Recompensa: -2.7029144935477554
Total de pasos: 2995844/3000000.0 - Recompensa: 0.16747020533865506
0.16747020533865506
Total de pasos: 2995845/3000000.0 - Recompensa: -0.47296619810660656
Total de pasos: 2995846/3000000.0 - Recompensa: -1.4812942504504623
Total de pasos: 2995847/3000000.0 - Recompensa: -0.17470262882903134
Total de pasos: 2995848/3000000.0 - Recompensa: 0.76339051732221
0.76339051732221
Total de pasos: 2995849/3000000.0 - Recompensa: -1.528381327928526
Total de pasos: 2995850/3000000.0 - Recompensa: -1.0441676482283242
Total de pasos: 2995851/3000000.0 - Recompensa: 0.4716361912349824
0.4716361912349824
Total de pasos: 2995852/3000000.0 - Recompensa: -1.0829299742378962
To

In [10]:
len (positive_rewards)

495982

#Step 3.2: Etapa de entrenamiento

In [11]:
def calc_memory_prob (total_steps, start_steps, initial_memory_prob):
    """
    Define una función para calcular la probabilidad de tomar una acción de la memoria de repetición en función
    del número total de pasos.

    Args:
        total_steps (int): El número total de pasos realizados en el entrenamiento.
        start_steps (int): El número de pasos antes de comenzar a utilizar la red de políticas en lugar de la
                           memoria de repetición.

    Returns:
        float: La probabilidad de tomar una acción de la memoria de repetición en el rango [0, 1].
    """
    if total_steps < start_steps:
        # Probabilidad decreciente de tomar una acción de la memoria de repetición a medida que total_steps aumenta
        return initial_memory_prob * (1 - total_steps / start_steps)
    else:
        # Probabilidad creciente de tomar una acción de la memoria de repetición a medida que total_steps supera start_steps
        return initial_memory_prob + (1 - initial_memory_prob) * (total_steps - start_steps) / (max_timesteps - start_steps)

def select_action (replay, policy, env, total_steps, start_steps, memory_prob, obs):
    """
      Sólo tenemos fase de explotación porque ya hemos rellenado la replaybuffer con transiciones positivas.

    * Explotación: Se refiere a aprovechar al máximo el conocimiento adquirido hasta el momento,
      es decir, elegir las acciones que el agente considera óptimas según su conocimiento actual.



    Args:
        policy      (TD3): plotica a entrenar
        env         (gym): entorno sobre el cual se aplica el modelo
        total_steps (int): El número total de pasos que ha realizado el agente.
        start_steps (int): El número de pasos iniciales antes de que el agente comience a utilizar su política.
        memory_prob (float): La probabilidad de tomar una acción de la experiencia del agente.
        obs (array): La observación actual del entorno.

    Returns:
        array: La acción seleccionada para ser ejecutada en el entorno.
    """
    #print (f'total_steps:{total_steps} - tart_steps:{start_steps}  ')
    if (total_steps < start_steps):
      (obs, next_obs, action, reward, done) = replay.sample_action (obs)
      # print (reward[0])
      action =  action [0]
    else:
      # Tomar una acción de la experiencia del agente (exploración)
      action = policy.select_action(np.array(obs))
      # Aplicar ruido a la acción para fomentar la exploración de forma más refinada
      current_explore_noise = max_explore_noise * max(1 - total_steps / max_timesteps, 0)
      if current_explore_noise != 0:
          action = (action + np.random.normal(0, max_explore_noise, size=action_dim)).clip(env.action_space.low, env.action_space.high)

    return action


# Creamos el objeto DDPG TD3 con los hiperparámetros definidos anteriormente
policy = TD3 (state_dim, action_dim, max_action,max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definirá la política a seguir por el agente


# replay_buff = SimplePrioritizedReplayBuffer()
replay_buff = replay_buff_pos
# Lista donde se guardarán las evaluaciones de la política durante el entrenamiento
evaluations = [evaluate_train_policy (policy, env)]

# Inicialización de variables utilizadas en el entrenamiento
total_steps        = 0     # Número total de pasos de entrenamiento que llevamos realizados
episode_steps      = 0     # Número de pasos realizados en el epidodeo
steps_since_eval   = 0     # Número de pasos desde la última evaluación de la política
episode_num        = 0     # Número de episodios completados durante el entrenamiento
done               = True  # Indica si el episodio actual ha finalizado, nos lo devuelve el entorno de gym

t0 = time.time()  # Tiempo inicial de referencia para medir el tiempo de entrenamiento
n_steps_epochs = []
all_rewards    = []
avg_losses_c   = []
avg_losses_a   = []
all_losses_c   = []
all_losses_a   = []
target_qs_c1   = []
target_qs_c2   = []
target_qs      = []
all_exploration_factor = []
############################################################################
# step 0: Bucle principal del entrenameinto de nuestro modelo.
##  REalizamos tantos pasos como indica el hiperparámetro "max_timesteps"
## A no ser que se estanque en un maximo local (mínimo) el humanoide aprenderá
## mejor cuantos mñas pasos realiza. Eta implementacición V2 utiliza una estrategia e-grady
## cuando finaliza el periodo de exploración. Es un entorno muy complicado con
## muchos grados de libertad y acciones infinitas por lo que es facil estancarse.
## Por ello, es muy importante el ruido aladido a las acciones.
#############################################################################

while total_steps < max_timesteps: # son pasos de tiempo
    # step 1: Se comprueba si el episodio ha concluido. Si lo ha hehco, realiamso el entrenamiento del modelo
    # con las acciones almacenadas en ReplayBuffer
    if done:
        # Comienza el entrenamiento del modelo si no es la primera iteración
        if total_steps != 0: #Ya que no se tendrá nada en la memoria de repetición
            print (f'Número de pasos del episodio {episode_num} son episode_steps:{episode_steps}')
            n_steps_epochs.append (episode_steps)
            # Realizamos el entrenamiento del modelo con las aciones almacenadas en ReplayBuffer
            rewards, losses_c,losses_a, target_qs_critic1, target_qs_critic2, target_qs = policy.train (replay_buff, episode_steps, batch_size, gamma,
                                                                               target_update_freq, policy_noise, noise_clip, policy_freq)
            #############
            # Nos guardamos las métricas del entrenamiento a estudio
            #############
            avg_reward = np.mean (rewards)
            avg_loss_c = np.mean (losses_c)
            avg_loss_a = np.mean (losses_a)
            # avg_expl_fact = np.mean(expl_fact)

            all_rewards.append (avg_reward)
            avg_losses_c.append (avg_loss_c)
            avg_losses_a.append (avg_loss_a)
            all_losses_c.append (losses_c.copy())
            all_losses_a.append (losses_a.copy())

            target_qs_c1.append (target_qs_critic1.copy())
            target_qs_c2.append (target_qs_critic2.copy())
            target_qs.append (target_qs.copy())

            # all_exploration_factor.append(avg_expl_fact)

            print("Total Steps: {} Episode Num: {} Reward: {} avg_loss_c: {} avg_loss_a: {}".format(total_steps, episode_num, episode_reward, avg_loss_c, avg_loss_a))


        # step 2: Se evalúa el rendimiento del episodio actual y se guarda la política
        # si se cumplen ciertas condiciones o criterios predefinidos.
        if steps_since_eval >= eval_frequency:
            steps_since_eval %= eval_frequency
            evaluations.append (evaluate_train_policy (policy, env))
            # Almacenamos el modelo en el estado de entrenamiento en el que se encuentra
            policy.save(file_model_name, directory="./pytorch_models")

            np.save("./results/%s" % (file_model_name), evaluations)

            save_env(file_model_name, directory="./results", env=env) # Guardamos el entorno en el estado actual
            # Serializar el ReplayBufferMemory y guardarlo en un archivo
            serialize_object(replay_buff, './results/replay_buffer_memory_v3_1.pickle')
            # Serializamos las métricas del entrenamiento
            lists_train_metrics = [all_rewards, avg_losses_c, avg_losses_a, all_losses_c, all_losses_a]
            attribute_names = ['rewards', 'losses']
            lists_to_serializable_object(lists_train_metrics, attribute_names, './results/serialized_list_train_metrics_v3_1.pickle')


        # step3: Reiniciar el entorno cuando finaliza el episodio de entrenamiento
        # Notar que según la logica del bucle, en el primer ciclo, este serña el primer paso.
        obs = env.reset()

        # Establecer "done" a Falso para parar el episodio
        done = False

        # Restablecer la recompensa del episodio y el contador de pasos del episodio
        episode_reward = 0
        episode_steps  = 0
        episode_num   += 1

    # step 4: Tomar acciones aleatorias antes de alcanzar el número de pasos iniciales
    start_steps = max_start_steps * max(1 - total_steps / max_timesteps, 0)         # Decaimiento lineal
    memory_prob = calc_memory_prob (total_steps, start_steps, initial_memory_prob)  # Probabilidad de tomar la acción en otro entorno
    action      = select_action (replay_buff, policy, env, total_steps, start_steps, memory_prob, obs)

    # step 5: El agente ejecuta una acción en el entorno, lo que resulta en una transición
    # de estado. Además, el agente recibe una recompensa del entorno como resultado
    # de su acción. Esta acción puede cambiar el estado del entorno y, por lo tanto,
    # influir en las futuras observaciones y recompensas del agente.
    #print (f' * nueva action:  -- tamaño:{len (action)}')
    new_obs, reward, done, _ = env.step (action)
    #print (f' * nuevo reward:{reward}')
    # Comprueba si el episodio ha terminado
    done_bool = 0 if episode_steps + 1 == max_episode_steps else float(done)

    # Aumenta la recompensa total del episodio
    episode_reward += reward

    # step 6: Almacenar nueva transición en el búfer de repetición de experiencias
    ## Hemos desestimado el buffer de repeticion con prioridad por el costo computacional
    replay_buff.add((obs, new_obs, action, reward, done_bool))

    #error_explora = 0.01 # añadimos un error muy bajo para asegurarnos que al menos se utiliza una vez
    #replay_buff.add(error_explora, (obs, new_obs, action, reward, done_bool))

    # Actualizar estado, tiempo de paso del episodio, tiempo total de pasos y pasos desde la última evaluación de la política
    obs = new_obs
    episode_steps    += 1
    total_steps      += 1
    steps_since_eval += 1

    # Llamamos a la funcion para crear graficas de entrenamiento
    create__metrics_imagen (evaluations, all_rewards,  avg_losses_c,avg_losses_a, all_losses_c,
                                                    all_losses_a, target_qs_c1,
                                                    target_qs_c2, target_qs,
                                                    n_steps_epochs, "v3_1", episode_num, total_steps)

tf = time.time()  # Tiempo final de referencia para medir el tiempo de entrenamiento
serialize_training (t0, tf, total_steps,"v3_1")
# Añadir la última actualización de la política a la lista de evaluaciones previa y guardar nuestro modelo
evaluations.append (evaluate_train_policy (policy, env))
if save_models:
    policy.save("gready_%s" % (file_model_name), directory="./pytorch_models")
np.save("./results/gready_%s" % (file_model_name), evaluations)



Se han truncado las últimas 5000 líneas del flujo de salida.
Total Steps: 531888 Episode Num: 8780 Reward: 85.05656993944457 avg_loss_c: 4.2711172657953185 avg_loss_a: -50.238874730929524
Número de pasos del episodio 8781 son episode_steps:99
Total Steps: 531987 Episode Num: 8781 Reward: 153.1438238487227 avg_loss_c: 4.0886034724688285 avg_loss_a: -50.65918635358714
Número de pasos del episodio 8782 son episode_steps:81
Total Steps: 532068 Episode Num: 8782 Reward: 111.63149037126396 avg_loss_c: 4.299364416687577 avg_loss_a: -50.469676076630016
Número de pasos del episodio 8783 son episode_steps:79
Total Steps: 532147 Episode Num: 8783 Reward: 82.68822679807445 avg_loss_c: 4.403769613821296 avg_loss_a: -50.349261078653456
Número de pasos del episodio 8784 son episode_steps:94
Total Steps: 532241 Episode Num: 8784 Reward: 116.1131736190497 avg_loss_c: 3.888538619305225 avg_loss_a: -50.37811206249481
Número de pasos del episodio 8785 son episode_steps:101
Total Steps: 532342 Episode Num:

# **Step 4:** Evaluación para extraeer videos de la politicca entrenada

In [ ]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque
#import mujoco_py
from TD3 import TD3, evaluate_policy, created_models_directory
import pybullet as p

# Iniciar el servidor de física de PyBullet
p.connect(p.DIRECT)



#######################################
###
######################################
env_name           = "HumanoidBulletEnv-v0"  # Nombre del entorno que vamos a entrena. Con forme se ha implementado el modelo TD3 poría ser cuaalquier entorno (preferiblemente de estados de acciones continuos)
seed               = 0        # Semilla utilizada para garantizar la reproducibilidad de los resultados
start_steps        = 1e4      # Número de iteraciones/timesteps antes de que el modelo comience a utilizar la red de políticas en lugar de elegir acciones al azar
eval_frequency     = 5e3      # Frecuencia de evaluación, es decir, cada cuántos pasos/timesteps se evalúa el desempeño del modelo
max_timesteps      = 5e6      # Número máximo de iteraciones/timesteps permitidos
save_models        = True     # Booleano que indica si se deben guardar los modelos pre-entrenados o no
explore_noise      = 0.1      # Desviación estándar del ruido gaussiano utilizado para la exploración
batch_size         = 100      # Tamaño del lote de muestras utilizadas en cada iteración de entrenamiento
gamma              = 0.99     # Factor de descuento gamma que afecta la importancia de las recompensas futuras en la función de pérdida
target_update_freq = 0.005    # Tasa de actualización para suavizar los parámetros de la red objetivo
policy_noise       = 0.2      # Desviación estándar del ruido gaussiano agregado a las acciones para promover la exploración
noise_clip         = 0.5      # Valor máximo permitido para el ruido gaussiano agregado a las acciones (política)
policy_freq        = 2        # Número de iteraciones entre actualizaciones de la red de políticas (modelo actor)

work_dir = os.path.join('exp', 'brs')
monitor_dir = os.path.join(work_dir, 'monitor')


file_model_name = created_models_directory (env_name, seed, save_models, "v3_1")
print ("---------------------------------------")
print ("Configuración: %s" % (file_model_name))
print ("---------------------------------------")

eval_episodes = 1000000
save_env_vid  = True

env = gym.make (env_name)
max_episode_steps = env._max_episode_steps
if save_env_vid:
  env = wrappers.Monitor (env, monitor_dir, force = True)
  # env = RecordEpisodeStatistics (env)
  env.reset ()
env.seed (seed)

torch.manual_seed (seed)
np.random.seed (seed)

state_dim  = env.observation_space.shape [0]
action_dim = env.action_space.shape [0]
max_action = float (env.action_space.high [0])

policy = TD3 (state_dim, action_dim, max_action,max_timesteps, initial_lr =  1e-4) # Notar que realmente el objeto entrenado definirá la política a seguir por el agente
policy.load (file_model_name, './pytorch_models/')
avg_reward, episode_rewards = evaluate_policy (policy, env, eval_episodes = eval_episodes)

# Graficar las recompensas por episodio
plt.plot(episode_rewards)
plt.xlabel('Episodio')
plt.ylabel('Recompensa')
plt.title('Recompensa por Episodio durante la Evaluación')

# Guardar la gráfica en un archivo de imagen (por ejemplo, en formato PNG)
plt.savefig('./results/recompensas_por_episodio_evaluacion_v3_1.png')

# Mostrar la gráfica en la ventana
plt.show()

---------------------------------------
Fichero de los modelos entrenados: TD3_HumanoidBulletEnv-v0_0_v3_1
---------------------------------------
---------------------------------------
Configuración: TD3_HumanoidBulletEnv-v0_0_v3_1
---------------------------------------
